In [1]:
import urllib
from sqlalchemy import create_engine, text
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime


# Connecting to SQL database and creating a table from df
odbc_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"
    "DATABASE=Financial_transaction;"
    "Trusted_Connection=yes;"
)

quoted = urllib.parse.quote_plus(odbc_str)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}")

# quick connection test
with engine.connect() as conn:
    print(conn.execute(text("SELECT 1")).scalar())

sql = """
SELECT * FROM AccountFeatures;
"""
df = pd.read_sql(text(sql), engine)
df.head()

InterfaceError: (pyodbc.InterfaceError) ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'Vineet\\Vineet Dadheech\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "Financial_transaction" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'Vineet\\Vineet Dadheech\'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "Financial_transaction" requested by the login. The login failed. (4060)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [ ]:
# checking for null values
df.isnull().sum()

In [ ]:
# creating customer tenure column
df['account_open_date'] = pd.to_datetime(df['account_open_date'])
df['customer_tenure_days'] = (datetime.now() - df['account_open_date']).dt.days
df['customer_tenure_years'] = df['customer_tenure_days'] / 365.25
df['customer_tenure_years']

In [ ]:
# create a new column age_group
labels = ['Young Adult', 'Adult', 'Middle-aged', 'Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels = labels)
df['age_group']

In [ ]:
df['total_fd_amount'] = pd.to_numeric(df['total_fd_amount'])
df.info()

In [ ]:
# creating a column to calculate total wealth
df['total_wealth'] = df['current_balance'] + df['total_fd_amount']
df[['account_id','total_wealth']]



In [ ]:
# creting a column to determine risky account holders for loan
df['outflow_to_balance_ratio'] = df['total_outflow'] / (df['current_balance'] + 1)
df[['account_id','outflow_to_balance_ratio']]